<a href="https://colab.research.google.com/github/kevinworthington/geospatial_colab/blob/main/Sentinel_Merge_Reduce_Multiple_GeoTiffs_by_Averaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following notebook searches for remote sensing data over a period of time and merges these images together by averaging the values of each of the pixels.

The preset boundary box is editable allowing you to choose where you'd like to do your analysis.

Referenced From: https://carpentries-incubator.github.io/geospatial-python/05-access-data.html

We'll start by installing the needed libraries, but if you have these already, skip to loading the libraries.

In [ ]:
%%capture
!pip install pystac_client

In [ ]:
%%capture
!pip install localtileserver

In [ ]:
%%capture
!pip install leafmap

In [ ]:
%%capture
!pip install rioxarray

In [ ]:
# Now load the required libraries, each includes their intended purpose in this project
from pystac_client import Client # To query STAC API endpoint
from shapely.geometry import shape,GeometryCollection,box # To work with drawn geometry
from shapely.geometry.polygon import Polygon
import leafmap # A feature-rich interactive map allow us to save drawn geometry (among other things)
import geojson # To parse geospatial data
import folium # Another interactive map
from shapely.ops import transform # The shapely transform module
import pyproj # A reprojection library
import rioxarray # To open raster images
from rioxarray import merge # To merge raster images
import numpy as np # To math raster images

In [ ]:
# Create a polygon to define our Area of Interest (AOI), this can be updated within our interactive map
drawn_box: dict = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -106.05059,
              39.856251
            ],
            [
              -106.05059,
              40.443559
            ],
            [
              -105.182217,
              40.443559
            ],
            [
              -105.182217,
              39.856251
            ],
            [
              -106.05059,
              39.856251
            ]
          ]
        ]
      }
    }
  ]
}

In [ ]:
# Create an interactive map allowing us to edit our initial AOI
# You can delete the existing polygon and/or draw mulitple polygons which will later be combined to create a boundary box used to search for data
m = leafmap.Map(center=[40, -100], zoom=4)
m.edit_vector(drawn_box)
m

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [ ]:
# save the drawn polygons
m.save_draw_features("data.geojson")

In [ ]:
# open the saved polygons. This is the only way to avoid copying and pasting the updated geojson
with open('data.geojson', 'r') as file:
    geojson_data = geojson.load(file)

In [ ]:
# Create a boudary box for all the drawn polygons

# Ref: rom https://gis.stackexchange.com/questions/270297/getting-bounding-boxes-for-all-polygons-in-geojson-feature-collection

# Define a list to store the geometry
polygons=[]
for f in geojson_data["features"]:
    polygon: Polygon = shape(f["geometry"])
    polygons.append(polygon)

# Create a colletion with all the geometries
polygons_gc=GeometryCollection(polygons)

# Store the bounds in a new variable
polygons_bbox= box(*polygons_gc.bounds)


In [ ]:
# Create a new interactive map with a boundary around all the shapes drawn earlier

# Create the map
folium_map = folium.Map([polygons_bbox.centroid.y,polygons_bbox.centroid.x], zoom_start=10)

# Add drawn boundaries
folium.GeoJson(geojson_data).add_to(folium_map)

# # Add max bounds
folium.GeoJson(polygons_bbox,
    style_function=lambda feature: {
        "fillColor": "#ff0000",
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
    }).add_to(folium_map)

folium_map

In [ ]:
# Connect to the STAC endpoint used to find satellite data
cmr_api_url = "https://earth-search.aws.element84.com/v1"#"https://cmr.earthdata.nasa.gov/stac/LPCLOUD"
client = Client.open(cmr_api_url)

# Run search
search = client.search(
    collections=["sentinel-2-l2a"],#["HLSL30.v2.0"],
    bbox=polygons_bbox.bounds,
    datetime="2021-03-01/2021-03-30",
) # nasa cmr cloud cover filtering is currently broken: https://github.com/nasa/cmr-stac/issues/239

# Retrieve search results
items = search.item_collection()
print(len(items))

18


In [ ]:
# Inspect the first item
items[0]

<Item id=S2A_13TDE_20210329_2_L2A>

In [ ]:
# visualize the first item
m = leafmap.Map(center=[polygons_bbox.centroid.y,polygons_bbox.centroid.x])
m.add_cog_layer(items[0].assets["nir"].href, name="")


m

Map(center=[40.149905000000004, -105.61640349999999], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [ ]:
# Sort the items by cloud_cover
items = search.item_collection()

items_sorted = sorted(items, key=lambda x: x.properties["eo:cloud_cover"])

In [ ]:
# Lets short list the items which we'll merge later and then clip
# Start by gathering the hrefs
item_hrefs=[]

for i in items_sorted[0:2]:
    item_hrefs.append(i.assets["nir"].href)

In [ ]:
# View the map file 'boundary'
# Use the file's lat and long column to center the map

folium_map = folium.Map([polygons_bbox.centroid.y,polygons_bbox.centroid.x], zoom_start = 10)

# Add the boundary
folium.GeoJson(polygons_bbox).add_to(folium_map)

for id, i in enumerate(item_hrefs):
      raster = rioxarray.open_rasterio(item_hrefs[id])
      # Create boundary boxes
      # use teh first item to create a reprojection transformer
      project = pyproj.Transformer.from_crs(items[0].properties["proj:epsg"], 4326, always_xy=True).transform

      bbox = box(*raster.rio.bounds())
      bbox_transformed = transform(project, bbox)

      folium.GeoJson(bbox_transformed,
          style_function=lambda feature: {
          "color": "purple",
      }).add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

# Merge the datasets

In [ ]:
# Start by creating a list of the images
nir_rasters=[]
for i in item_hrefs:
    nir_rasters.append(rioxarray.open_rasterio(i, masked=True))


In [ ]:
# Set our boundry to the CRS of the raster so we can use it to clip
project_from_4326 = pyproj.Transformer.from_crs(4326,items[0].properties["proj:epsg"], always_xy=True).transform
polygons_bbox_transformed = transform(project_from_4326, polygons_bbox)

In [ ]:
# create a custom merge function allowing us to average the overlapping pixcels and merger
def custom_merge_avg(old_data, new_data, old_nodata, new_nodata, index=None, roff=None, coff=None):
    old_data[:] = np.nanmean( np.array([ old_data, new_data ]), axis=0 )

nir_merged = merge.merge_arrays(nir_rasters,polygons_bbox_transformed.bounds,method = custom_merge_avg)

In [ ]:
# save our merged geotiff
nir_merged.rio.to_raster("nir_merged.tif")

In [ ]:
satellite = leafmap.download_file("nir_merged.tif","nir_merged.tif")
m = leafmap.Map()
m.add_raster(satellite)
m

nir_merged.tif already exists. Skip downloading. Set overwrite=True to overwrite.


Map(center=[40.1499185, -105.62018850000001], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…